## 意图识别简介

*Copyright (c) 2023 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

自然语言处理（Natural Language Processing, NLP）是一种机器学习技术，使计算机具有解释、理解和使用人类语言的能力。现在的各类政企拥有大量的语音和文本数据，这些数据来自各种通信渠道，如电子邮件、文本信息、社交媒体新闻报道、视频、音频等等。他们使用NLP软件来自动处理这些数据，分析信息中的意图或情绪，并实时回应人们的沟通。

意图识别是自然语言处理中的基础任务之一，在搜索引擎、智能客服、机器人等产品中都有着重要的应用。

这里，我们使用 BERT-QTC [1] 这一量子经典混合模型来实现意图识别任务。这里，我们的意图识别任务是针对输入的文本，确定这句话所对应的意图，如聊天、询问菜谱、询问电视节目等。

我们使用 [SMP2017 数据集](https://github.com/HITlilingzhi/SMP2017ECDT-DATA) [2]来进行实验展示，我们选取了其中七个类别来进行训练，分别是火车，音乐，天气，短信，电话，航班，新闻。数据的样本如下：

- 查宁波到北京的火车票
- 我想知道浙江义乌的天气
- 帮我查一下明天广州到长沙的航班
- 我想听最新军事新闻。

## 使用 BERT-QTC 实现意图识别

BERT-QTC模型是一个量子经典混合模型。它的模型结构如下：

![the arch of the bert-qtc model](bert_qtc_arch.png)

模型的具体流程如下：

1. 使用 BERT [3] 对输入文本进行特征提取，得到句子级别的特征表示。
2. 对于 BERT 提取到的特征，使用量子时序卷积（Quantum Temporal Convolution, QTC）和全局最大池化（Global Maxing Pooling, GMP）进行进一步的特征提取和降维。
3. 使用全连接层，进行分类预测，得到分类结果。

### 工作流

BERT-QTC 模型是学习类的模型。我们需要先使用数据集对模型进行训练。在训练收敛后，我们便得到了一个训练好的模型，这个模型可以对这类数据进行分类。其中，由于 BERT 模型是一个大型语言模型。因此，我们使用预训练好的模型来进行特征提取。在之后的模型训练过程中，BERT 部分的参数不再进行训练。

总结来说，其工作流如下：

1. 制备意图识别的数据集。
2. 使用数据集对 BERT-QTC 模型训练，得到训练好的模型。
3. 使用该模型对输入的文本进行预测，得到预测结果。

## 如何使用

这里，我们给出一个训练好的模型供测试使用。只需要在 `example.toml` 这个配置文件中进行对应的配置。然后输入 `python intent_classification.py --config example.toml` 即可对输入的文本进行测试。

### 在线演示

这里，我们给出一个在线演示的版本。首先定义配置文件的内容：

In [1]:
test_toml = r"""
task = 'test'
text = '查宁波到北京的火车票'
num_filter = 1
kernel_size = 5
circuit_depth = 2
padding = 2
model_path = 'decoder.pdparams'
bert_model = 'bert-base-chinese'
hidden_size = 768
classes = ['火车', '音乐', '天气', '短信', '电话', '航班', '新闻']
"""

接下来是测试部分的代码：

In [3]:
import os
import warnings

warnings.filterwarnings('ignore')
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

import toml
from paddle_quantum.qml.bert_qtc import train, inference

config = toml.loads(test_toml)
task = config.pop('task')
if task == 'train':
    train(**config)
elif task == 'test':
    prediction = inference(**config)
    text = config['text']
    print(f'输入的文本是：{text}。')
    print(f'模型的预测结果是：{prediction}。')
else:
    raise ValueError("未知的任务，它可以是'train'或'test'。")

输入的文本是：查宁波到北京的火车票。
模型的预测结果是：火车。


在这里，我们只需要修改要配置文件中的 text 的内容，再运行整个代码，就可以快速对其它文本测试。

## 注意事项

在这里，我们提供的仅仅是一个demo模型，其效果只用来展示。对于实际应用场景来说，需要进行针对性的设计和训练，才能达到更好的效果。

### 数据集结构

如果想要使用自定义数据集进行训练，只需要按照规则来准备数据集即可。在数据集文件夹中准备 `train.txt` 和 `test.txt`，如果需要验证集的话还有 `dev.txt`。每个文件里使用一行代表一条数据。每行内容包含文本和对应的标签，使用制表符隔开。文本是由空格隔开的文字组成。

### 配置文件介绍

在 `test.toml` 里有测试所需要的完整的配置文件内容参考。在 `train.toml` 里有训练所需要的完整的配置文件内容参考。使用 `python intent_classification --config train.toml` 可以对模型进行训练。使用 `python intent_classification --config test.toml` 可以加载训练好的模型进行测试。

## 参考文献

[1] Yang C H H, Qi J, Chen S Y C, et al. When BERT meets quantum temporal convolution learning for text classification in heterogeneous computing[C]//ICASSP 2022-2022 IEEE International Conference on Acoustics, Speech and Signal Processing (ICASSP). IEEE, 2022: 8602-8606.

[2] Zhang W N, Chen Z, Che W, et al. The first evaluation of Chinese human-computer dialogue technology[J]. arXiv preprint arXiv:1709.10217, 2017.

[3] Devlin J, Chang M W, Lee K, et al. BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding[C]//Proceedings of the 2019 Conference of the North American Chapter of the Association for Computational Linguistics: Human Language Technologies, Volume 1 (Long and Short Papers). 2019: 4171-4186.